In [106]:
import pandas as pd
import math
import os
from datetime import datetime

In [107]:
# Constants
initial_rating = 1500
initial_rd = 350
tau = 0.5
q = math.log(10) / 400
c = 1.5

# Glicko Functions
def g(rd):
    return 1 / (1 + 3 * (q**2) * (rd**2) / (math.pi**2))**0.5

def E(r, ri, rdi):
    return 1 / (1 + 10**(g(rdi) * (ri - r) / 400))

def d2(r, ri, rdi):
    inner = (g(rdi)**2) * E(r, ri, rdi) * (1 - E(r, ri, rdi))

    try:
        return 1 / ((q**2) * inner)
    except ZeroDivisionError as e:
        print(r, ri, rdi)
        raise e

def new_rd(rd, d2):
    return 1 / ((1 / rd**2) + (1 / d2))**0.5

def new_rating(r, rd, ri, rdi, s, d2):
    return r + (q / ((1 / rd**2) + (1 / d2))) * g(rdi) * (s - E(r, ri, rdi))

# Function to increase RD over time due to inactivity
def increase_rd_over_time(player_ratings, current_time):
    for player, (rating, rd, last_active) in player_ratings.items():
        # Calculate time elapsed in days
        days_inactive = (current_time - last_active).days
        if days_inactive > 0:
            # Increase RD based on time elapsed
            new_rd = min(math.sqrt(rd**2 + c**2 * days_inactive), initial_rd)
            player_ratings[player] = (rating, new_rd, last_active)

def update_ratings_doubles(player_ratings, match_results, current_time):
    increase_rd_over_time(player_ratings, current_time)
    for match in match_results:
        player1, player2, player3, player4, result = match
        r1, rd1, _ = player_ratings.get(player1, (initial_rating, initial_rd, current_time))
        r2, rd2, _ = player_ratings.get(player2, (initial_rating, initial_rd, current_time))
        r3, rd3, _ = player_ratings.get(player3, (initial_rating, initial_rd, current_time))
        r4, rd4, _ = player_ratings.get(player4, (initial_rating, initial_rd, current_time))
        
        # pretend teams are 1 player, take average rating
        r12 = (r1 + r2) / 2
        r34 = (r3 + r4) / 2

        # mean squared rating deviations
        # rd12 = (rd1 * rd1 + rd2 * rd2) ** 0.5
        # rd34 = (rd3 * rd3 + rd4 * rd4) ** 0.5
        rd12 = (rd1**2 + rd2**2) ** 0.5
        rd34 = (rd3**2 + rd4**2) ** 0.5

        d2_12 = d2(r12, r34, rd34)
        d2_34 = d2(r34, r12, rd12)
        
        prop1 = rd1 / (rd1 + rd2)
        prop2 = rd2 / (rd1 + rd2)
        prop3 = rd3 / (rd3 + rd4)
        prop4 = rd4 / (rd3 + rd4)

        new_rd12 = new_rd(rd12, d2_12)
        new_rd34 = new_rd(rd34, d2_34)

        new_r12 = new_rating(r12, rd12, r34, rd34, result, d2_12)
        new_r34 = new_rating(r34, rd34, r12, rd12, 1 - result, d2_34)

        dr12 = (new_r12 - r12) * 2
        dr34 = (new_r34 - r34) * 2
        drd12 = (new_rd12 - rd12) * 2
        drd34 = (new_rd34 - rd34) * 2
        player_ratings[player1] = (r1 + prop1 * dr12, rd1 + rd1 / rd12 * drd12, current_time)
        player_ratings[player2] = (r2 + prop2 * dr12, rd2 + rd2 / rd12 * drd12, current_time)
        player_ratings[player3] = (r3 + prop3 * dr34, rd3 + rd3 / rd34 * drd34, current_time)
        player_ratings[player4] = (r4 + prop4 * dr34, rd4 + rd4 / rd34 * drd34, current_time)


players = {}
matches = [("player1", "player2", "player3", "player4", 1), ("player4", "player3", "player2", "player1", 0)]
update_ratings_doubles(players, matches, datetime.now())
print(players)


{'player1': (1775.1950034485953, 177.02405563845807, datetime.datetime(2024, 6, 3, 1, 15, 37, 10787)), 'player2': (1775.1950034485953, 177.02405563845807, datetime.datetime(2024, 6, 3, 1, 15, 37, 10787)), 'player3': (1224.8049965514047, 177.02405563845807, datetime.datetime(2024, 6, 3, 1, 15, 37, 10787)), 'player4': (1224.8049965514047, 177.02405563845807, datetime.datetime(2024, 6, 3, 1, 15, 37, 10787))}


In [108]:
tf = pd.read_csv('data/tournaments_wtt.tsv', sep='\t', parse_dates=['StartDateTime']).sort_values(['StartDateTime'])

In [109]:
directory = os.fsencode('data/wtt_cleaned/matches')

players = {}

for event in tf.itertuples():
    if not os.path.isfile(f'data/wtt_cleaned/matches/{event.EventId}.tsv'):
        continue

    df = pd.read_csv(f'data/wtt_cleaned/matches/{event.EventId}.tsv', sep='\t', parse_dates=['start'])
    # event_id	doc	fmt	gender	stage	stage_id	duration	start	a_id	b_id	x_id	y_id	res_a	res_x
    # 2345	TTEMTEAM--------------FNL-00010001--------	T	M	FNL	00010001	1560	2021-08-06 10:34:29	110267	105649	102832	101222	3	0
    matches = []
    for row in df[df.fmt == 'D'].itertuples():
        matches.append((row.a_id, row.b_id, row.x_id, row.y_id, int(row.res_a > row.res_x)))        
    update_ratings_doubles(players, matches, event.StartDateTime)

resf = pd.DataFrame.from_dict(players, orient='index')
resf

,0,1,2
117820.0,1329.858972,111.322644,2024-05-01
110752.0,1620.131052,250.206438,2022-08-01
114706.0,1089.842697,29.406287,2024-05-20
114105.0,1219.837297,47.466548,2024-05-01
121582.0,1687.464227,55.708843,2024-05-01
...,...,...,...
205717.0,1152.949151,151.315165,2024-05-20
144705.0,1340.106656,72.297230,2024-05-21
136056.0,1340.106656,72.297230,2024-05-21
144795.0,1352.574511,190.546845,2024-05-21


In [110]:
pf = pd.read_csv('data/wtt_cleaned/players.tsv', sep='\t').set_index('id')

In [111]:
rf = resf.join(pf)
srf = rf.sort_values(0, ascending=False)

In [112]:
w100 = srf[
    #(srf.gender == 'W') &
    (srf[1] <= 100)
].head(100)
w100[0] = w100[0].round().astype('int')
w100[1] = w100[1].round().astype('int')
w100.reset_index(inplace=True)
w100.drop(columns=['index', 'gender', 2], inplace=True)
w100.reset_index(inplace=True)
w100.rename(columns={'index': 'rank', 0: 'rating', 1: 'conf'}, inplace=True)
w100['rank'] += 1
w100.to_csv('top100D.tsv', sep='\t', index=False)

In [113]:
with open('top100d.txt', 'w') as f:
    f.write('|rank|rating|±dev|org|name|\n')
    f.write('|:-|:-|:-|:-|:-|\n')
    for row in w100.set_index('rank').itertuples():
        f.write('|'+ '|'.join((str(r) for r in row)) + '|\n')

In [104]:
directory = os.fsencode('data/wtt_cleaned/matches')

players = {}

events = []
for event in tf.itertuples():
    if not os.path.isfile(f'data/wtt_cleaned/matches/{event.EventId}.tsv'):
        continue

    events.append(
        pd.read_csv(f'data/wtt_cleaned/matches/{event.EventId}.tsv', sep='\t', parse_dates=['start'])
    )

matchf = pd.concat(events)
matchf

,event_id,doc,fmt,gender,stage,stage_id,duration,start,a_id,b_id,x_id,y_id,res_a,res_x
0,2410,TTEMSINGLES-----------FNL-000100--,S,M,FNL,100,2884,2021-03-06 11:40:00,107028,NaN,121582,NaN,4,1
1,2410,TTEMSINGLES-----------RND2001100----------,S,M,RND2,1100,1225,2021-03-01 08:20:00,107445,NaN,109002,NaN,3,0
2,2410,TTEWDOUBLES-----------FNL-000100--,D,W,FNL,100,1481,2021-03-06 07:00:00,117820,110752.0,114706,114105.0,3,0
3,2410,TTEWSINGLES-----------FNL-000100--,S,W,FNL,100,3464,2021-03-06 11:00:00,117821,NaN,123672,NaN,4,2
4,2410,TTEXDOUBLES-----------FNL-000100--,D,X,FNL,100,3201,2021-03-06 08:20:00,121582,110797.0,105136,137894.0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2865,TTEWSINGLES-----------RND1000400----------,S,W,RND1,400,1512,2024-05-22 05:45:00,136403,NaN,124363,NaN,3,0
102,2865,TTEWSINGLES-----------RND1000200----------,S,W,RND1,200,2834,2024-05-21 05:20:00,122716,NaN,137465,NaN,2,3
103,2865,TTEMSINGLES-----------RND1001100----------,S,M,RND1,1100,2561,2024-05-21 03:35:00,145550,NaN,133893,NaN,3,2
104,2865,TTEMSINGLES-----------RND2000300----------,S,M,RND2,300,1204,2024-05-22 06:20:00,117345,NaN,132312,NaN,3,0


In [114]:
def print_matches(id):
    xyb = matchf[(matchf.fmt == 'D') & ((matchf.a_id == id) | (matchf.b_id == id) | (matchf.x_id == id) | (matchf.y_id == id))]
    wnames = (
        xyb.merge(pf[['name']]
        .rename(columns={'name': 'name_a'}), left_on='a_id', right_index=True)
        .merge(pf[['name']]
        .rename(columns={'name': 'name_b'}), left_on='b_id', right_index=True)
        .merge(pf[['name']]
        .rename(columns={'name': 'name_x'}), left_on='x_id', right_index=True)
        .merge(pf[['name']]
        .rename(columns={'name': 'name_y'}), left_on='y_id', right_index=True)
    )
    wnames[['res_a', 'res_x', 'name_a', 'name_b', 'name_x', 'name_y']].to_csv('tmp.tsv', sep='\t', index=False)

In [118]:
print_matches(112019)